In [1]:
from Helper.Load.load import Base_Load
from Helper.Load.supervised import Supervised
from Helper.Load.regression import Regression
from Helper.Load.classification import Classification
from Helper.Visual.Visual import Visual
from Helper.Visual.classification import VClassification
from Helper.Transform.Transform.classification import Classificaton_Transform
from Helper.Train.Train import Train

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
loadC = Classification("Data", "led7digit.dat", "led7digit-names.txt")
transformC = Classificaton_Transform(loadC)
trainC = Train(loadC, Classificaton_Transform, RandomForestClassifier)

> /Users/jacob/Desktop/Project/MachineLearning/KEEL/Projects/Supervised/Standard/led7digit/Helper/Transform/Transform/classification.py(32)visual_data()
-> return pd.DataFrame(self.featureU().transform(data), columns=data.columns)


(Pdb)  c


> /Users/jacob/Desktop/Project/MachineLearning/KEEL/Projects/Supervised/Standard/led7digit/Helper/Transform/Transform/classification.py(32)visual_data()
-> return pd.DataFrame(self.featureU().transform(data), columns=data.columns)


(Pdb)  c


In [3]:
loadC.data.head()
transformC.data.head()

,Led1,Led2,Led3,Led4,Led5,Led6,Led7,Number
0,1,0,1,1,0,1,1,3
1,1,1,1,0,0,1,0,4
2,0,1,1,1,1,1,1,8
3,1,0,1,0,0,1,0,7
4,1,1,1,0,1,1,1,0


In [4]:
pipe = trainC.pipeline

In [5]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'pipeline', 'estimator', 'pipeline__memory', 'pipeline__steps', 'pipeline__inputs', 'pipeline__inputs__n_jobs', 'pipeline__inputs__transformer_list', 'pipeline__inputs__transformer_weights', 'pipeline__inputs__Led1', 'pipeline__inputs__Led2', 'pipeline__inputs__Led3', 'pipeline__inputs__Led4', 'pipeline__inputs__Led5', 'pipeline__inputs__Led6', 'pipeline__inputs__Led7', 'pipeline__inputs__Led1__memory', 'pipeline__inputs__Led1__steps', 'pipeline__inputs__Led1__Led1_selector', 'pipeline__inputs__Led1___LabelBinarizer_transform', 'pipeline__inputs__Led1__Led1_selector__key', 'pipeline__inputs__Led2__memory', 'pipeline__inputs__Led2__steps', 'pipeline__inputs__Led2__Led2_selector', 'pipeline__inputs__Led2___LabelBinarizer_transform', 'pipeline__inputs__Led2__Led2_selector__key', 'pipeline__inputs__Led3__memory', 'pipeline__inputs__Led3__steps', 'pipeline__inputs__Led3__Led3_selector', 'pipeline__inputs__Led3___LabelBinarizer_transform', 'pipeline__inputs__Led

In [6]:
from Data.hyperparameters import hyperparameters

In [7]:
hyperparameters

{'estimator__max_depth': [1, 2, 4, 8, 10],
 'estimator__n_estimators': [100, 110, 120, 200, 250, 300],
 'estimator__class_weight': ['balanced']}

In [8]:
clf = trainC.construct_GSCV(hyperparameters)
clf.fit(loadC.partition.X_train, loadC.partition.y_train.values.ravel())

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pipeline', Pipeline(memory=None,
     steps=[('inputs', FeatureUnion(n_jobs=None,
       transformer_list=[('Led1', Pipeline(memory=None,
     steps=[('Led1_selector', NumberSelector(key='Led1')), ('_LabelBinarizer_transform', <Helper.Transform.Custom.nominal._LabelBinarizer object at 0x1a2...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'estimator__max_depth': [1, 2, 4, 8, 10], 'estimator__n_estimators': [100, 110, 120, 200, 250, 300], 'estimator__class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
clf.best_params_

{'estimator__class_weight': 'balanced',
 'estimator__max_depth': 2,
 'estimator__n_estimators': 110}

In [10]:
clf.refit

True

In [11]:
preds = clf.predict(loadC.partition.X_test)
probs = clf.predict_proba(loadC.partition.X_test)
np.mean(preds == loadC.partition.y_test.values.flatten())

0.7272727272727273

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(loadC.partition.y_test.values, preds)

array([[11,  0,  0,  0,  0,  0,  1,  0,  1,  2],
       [ 0,  9,  0,  1,  0,  0,  0,  2,  0,  0],
       [ 0,  0, 13,  3,  0,  0,  1,  0,  0,  0],
       [ 0,  1,  0, 15,  0,  1,  0,  0,  0,  2],
       [ 0,  2,  0,  0, 12,  1,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2, 14,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1, 13,  0,  2,  0],
       [ 1,  2,  2,  0,  1,  1,  0, 12,  0,  0],
       [ 1,  0,  2,  1,  0,  1,  2,  0,  9,  1],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  3, 12]])